<a href="https://colab.research.google.com/github/Ultimo15/Gu-charo-Activo-Predictor/blob/main/Guacharo_PRO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Script Forense para analizar Cóndor Gana y buscar datos pre-cargados
# Ejecutar en Google Colab

!pip install selenium webdriver-manager beautifulsoup4 requests
!apt-get update
!apt-get install chromium-chromedriver

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import json
import re
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta

# Configurar Chrome
def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    # Removing performance logging options as they seem to cause issues
    # chrome_options.add_argument('--enable-logging')
    # chrome_options.add_argument('--log-level=0')
    # chrome_options.add_experimental_option('perfLoggingPrefs', {
    #     'enableNetwork': True,
    #     'enablePage': False,
    # })
    # chrome_options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})

    driver = webdriver.Chrome(options=chrome_options)
    return driver

print("🔍 ANÁLISIS FORENSE DE CÓNDOR GANA")
print("=" * 50)

# 1. ANÁLISIS DEL CÓDIGO FUENTE HTML
print("\n1️⃣ ANALIZANDO CÓDIGO FUENTE HTML...")
driver = setup_driver()
driver.get("https://tripletachira.com/condor.php")
time.sleep(3)

html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')

# Buscar datos en comentarios HTML
print("\n📝 Buscando en comentarios HTML...")
comments = soup.find_all(string=lambda text: isinstance(text, str) and text.strip().startswith('<!--'))
for comment in comments:
    if any(keyword in comment.lower() for keyword in ['resultado', 'numero', 'ganador', 'premio', 'sorteo']):
        print(f"⚠️ Comentario sospechoso encontrado: {comment[:100]}...")

# 2. ANÁLISIS DE SCRIPTS JAVASCRIPT
print("\n2️⃣ ANALIZANDO SCRIPTS JAVASCRIPT...")
scripts = soup.find_all('script')
suspicious_patterns = []

for i, script in enumerate(scripts):
    if script.string:
        content = script.string

        # Buscar arrays o objetos con datos
        array_pattern = r'(?:var|let|const)\s+(\w+)\s*=\s*\[(.*?)\]'
        object_pattern = r'(?:var|let|const)\s+(\w+)\s*=\s*\{(.*?)\}'

        arrays = re.findall(array_pattern, content, re.DOTALL)
        objects = re.findall(object_pattern, content, re.DOTALL)

        # Buscar fechas futuras
        future_dates = []
        date_patterns = [
            r'\d{4}-\d{2}-\d{2}',
            r'\d{2}/\d{2}/\d{4}',
            r'\d{2}-\d{2}-\d{4}'
        ]

        for pattern in date_patterns:
            dates = re.findall(pattern, content)
            for date_str in dates:
                try:
                    # Intentar parsear la fecha
                    if '-' in date_str and len(date_str.split('-')[0]) == 4:
                        date_obj = datetime.strptime(date_str, '%Y-%m-%d')
                    elif '/' in date_str:
                        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
                    else:
                        date_obj = datetime.strptime(date_str, '%d-%m-%Y')

                    if date_obj > datetime.now():
                        future_dates.append(date_str)
                except:
                    pass

        if future_dates:
            print(f"\n⚠️ Script #{i} contiene fechas futuras: {future_dates}")

        # Buscar números que podrían ser resultados
        number_patterns = re.findall(r'\b\d{2,3}\b', content)
        if len(number_patterns) > 5:
            print(f"\n🔢 Script #{i} contiene múltiples números: {number_patterns[:10]}...")

        # Buscar palabras clave
        keywords = ['resultado', 'ganador', 'premio', 'numero', 'sorteo', 'condor', 'futuro', 'proximo']
        for keyword in keywords:
            if keyword in content.lower():
                context = content[max(0, content.lower().find(keyword)-50):content.lower().find(keyword)+50]
                print(f"\n🔑 Palabra clave '{keyword}' encontrada en Script #{i}:")
                print(f"   Contexto: ...{context}...")

# 3. ANÁLISIS DE ALMACENAMIENTO LOCAL
print("\n3️⃣ ANALIZANDO ALMACENAMIENTO LOCAL...")
storage_check = """
var results = {
    localStorage: {},
    sessionStorage: {},
    cookies: document.cookie,
    indexedDB: []
};

// Revisar localStorage
for(var i = 0; i < localStorage.length; i++) {
    var key = localStorage.key(i);
    results.localStorage[key] = localStorage.getItem(key);
}

// Revisar sessionStorage
for(var i = 0; i < sessionStorage.length; i++) {
    var key = sessionStorage.key(i);
    results.sessionStorage[key] = sessionStorage.getItem(key);
}

return results;
"""

storage_data = driver.execute_script(storage_check)
print("\n📦 Datos en almacenamiento local:")
print(f"   - localStorage: {storage_data['localStorage']}")
print(f"   - sessionStorage: {storage_data['sessionStorage']}")
print(f"   - Cookies: {storage_data['cookies']}")

# 4. ANÁLISIS DE PETICIONES DE RED
print("\n4️⃣ ANALIZANDO PETICIONES DE RED...")
# Removing the problematic get_log call here
# logs = driver.get_log('performance')
network_requests = []

# The previous logs could not be retrieved, skipping the analysis of network requests from logs.
# for log in logs:
#     try:
#         message = json.loads(log['message'])
#         if 'Network.requestWillBeSent' in message['message']['method']:
#             request = message['message']['params']['request']
#             url = request['url']

#             # Filtrar URLs interesantes
#             if any(keyword in url.lower() for keyword in ['api', 'ajax', 'data', 'result', 'json', 'condor']):
#                 network_requests.append({
#                     'url': url,
#                     'method': request['method'],
#                     'headers': request.get('headers', {})
#                 })
#     except:
#         pass

print("\n🌐 Peticiones de red interesting: (Skipped due to logging issue)")
# for req in network_requests:
#     print(f"   - {req['method']} {req['url']}")


# 5. BUSCAR DATOS EN ATRIBUTOS HTML
print("\n5️⃣ BUSCANDO DATOS EN ATRIBUTOS HTML...")
elements_with_data = soup.find_all(attrs={"data-resultado": True})
elements_with_data.extend(soup.find_all(attrs={"data-number": True}))
elements_with_data.extend(soup.find_all(attrs={"data-premio": True}))

for elem in elements_with_data:
    print(f"\n⚠️ Elemento con datos: {elem.name}")
    for attr, value in elem.attrs.items():
        if 'data' in attr:
            print(f"   - {attr}: {value}")

# 6. ANÁLISIS DE VARIABLES GLOBALES JS
print("\n6️⃣ ANALIZANDO VARIABLES GLOBALES JAVASCRIPT...")
global_vars = """
var suspicious = [];
for(var prop in window) {
    if(window.hasOwnProperty(prop)) {
        var value = window[prop];
        if(prop.toLowerCase().includes('result') ||
           prop.toLowerCase().includes('ganador') ||
           prop.toLowerCase().includes('numero') ||
           prop.toLowerCase().includes('condor') ||
           prop.toLowerCase().includes('premio')) {
            suspicious.push({name: prop, value: value});
        }
    }
}
return suspicious;
"""

suspicious_globals = driver.execute_script(global_vars)
if suspicious_globals:
    print("\n🚨 Variables globales sospechosas encontradas:")
    for var in suspicious_globals:
        print(f"   - {var['name']}: {str(var['value'])[:100]}...")

# 7. BUSCAR ENDPOINTS OCULTOS
print("\n7️⃣ BUSCANDO ENDPOINTS Y APIs OCULTAS...")
# Buscar en el código fuente
api_patterns = [
    r'["\']([^"\']*api[^"\']*)["\']',
    r'["\']([^"\']*\.php[^"\']*)["\']',
    r'["\']([^"\']*\.json[^"\']*)["\']',
    r'url\s*:\s*["\']([^"\']+)["\']',
    r'fetch\s*\(["\']([^"\']+)["\']'
]

found_endpoints = set()
for pattern in api_patterns:
    matches = re.findall(pattern, html_source, re.IGNORECASE)
    found_endpoints.update(matches)

print("\n🔗 Posibles endpoints encontrados:")
for endpoint in found_endpoints:
    if not any(skip in endpoint for skip in ['google', 'facebook', 'jquery', 'bootstrap']):
        print(f"   - {endpoint}")

# 8. INTENTAR ACCEDER A ENDPOINTS COMUNES
print("\n8️⃣ PROBANDO ENDPOINTS COMUNES...")
base_url = "https://tripletachira.com/"
common_endpoints = [
    "api/condor",
    "api/resultados",
    "api/condor/resultados",
    "data/condor.json",
    "resultados/condor.json",
    "condor_data.php",
    "get_resultados.php",
    "ajax/condor.php"
]

for endpoint in common_endpoints:
    try:
        response = requests.get(base_url + endpoint, timeout=5)
        if response.status_code == 200:
            print(f"\n✅ ENDPOINT ACTIVO: {endpoint}")
            print(f"   Respuesta: {response.text[:200]}...")
    except:
        pass

# 9. ANÁLISIS FINAL
print("\n" + "="*50)
print("📊 RESUMEN DEL ANÁLISIS:")
print("="*50)

# Removed the problematic line attempting to get console data
# console_data = driver.execute_script("return window.performance.getEntries();")
# print(f"\n📈 Total de recursos cargados: {len(console_data)}")

# Buscar patrones numéricos sospechosos en todo el HTML
all_numbers = re.findall(r'\b\d{2,3}\b', html_source)
number_groups = {}
for num in all_numbers:
    if num not in number_groups:
        number_groups[num] = 0
    number_groups[num] += 1

print("\n🔢 Números más frecuentes en la página:")
sorted_numbers = sorted(number_groups.items(), key=lambda x: x[1], reverse=True)
for num, count in sorted_numbers[:10]:
    if count > 1:
        print(f"   - {num}: aparece {count} veces")

driver.quit()

print("\n✅ Análisis completado.")
print("\n💡 RECOMENDACIONES:")
print("1. Revisa los endpoints encontrados manualmente")
print("2. Los datos podrían estar cifrados en las variables JavaScript")
print("3. Monitorea las peticiones de red al cambiar fechas (if possible with a different logging setup)")
print("4. Algunos resultados podrían cargarse dinámicamente via WebSocket")